In [51]:
import torch
import torch.nn as nn
import torch.nn.functional as func
import torch.nn.init as torch_init
#import torch.optim as optim

class ModulationPredictionCNN(nn.Module):
    """ A one dimensional convolutional neural network model.

    Consists of six Conv1d layers, followed by max pooling
    and one fully-connected (FC) layer:

    conv1 -> conv2 -> conv3 -> conv4 con5 -> conv6 -> fc  (outputs)

    Make note:
    - Inputs are expected to be 2 channel signals of length 1024

    """

    def __init__(self):
        super(ModulationPredictionCNN, self).__init__()
        k = 8
        # conv1
        in_dim = 2
        out_dim = 16
        self.conv1 = nn.Conv1d(in_channels=in_dim, out_channels=out_dim, kernel_size=k, padding=4)
        self.conv1_norm = nn.BatchNorm1d(out_dim)
        self.pool1 = nn.MaxPool1d(kernel_size=2, stride=2)  
        
        # conv2
        in_dim = out_dim
        out_dim = 24
        self.conv2 = nn.Conv1d(in_channels=in_dim, out_channels=out_dim, kernel_size=k, padding=4)
        self.conv2_norm = nn.BatchNorm1d(out_dim)
        self.pool2 = nn.MaxPool1d(kernel_size=2, stride=2)
        
        #conv3
        in_dim = out_dim
        out_dim = 36
        self.conv3 = nn.Conv1d(in_channels=in_dim, out_channels=out_dim, kernel_size=k, padding=4)
        self.conv3_norm = nn.BatchNorm1d(out_dim)
        self.pool3 = nn.MaxPool1d(kernel_size=2, stride=2)
        
        #conv4
        in_dim = out_dim
        out_dim = 48
        self.conv4 = nn.Conv1d(in_channels=in_dim, out_channels=out_dim, kernel_size=k, padding=4)
        self.conv4_norm = nn.BatchNorm1d(out_dim)
        self.pool4 = nn.MaxPool1d(kernel_size=2, stride=2)
        
        #conv5
        in_dim = out_dim
        out_dim = 64
        self.conv5 = nn.Conv1d(in_channels=in_dim, out_channels=out_dim, kernel_size=k, padding=4)
        self.conv5_norm = nn.BatchNorm1d(out_dim)
        self.pool5 = nn.MaxPool1d(kernel_size=2, stride=2)
        
        #conv6
        in_dim = out_dim
        out_dim = 96
        self.conv6 = nn.Conv1d(in_channels=in_dim, out_channels=out_dim, kernel_size=k, padding=4)
        self.conv6_norm = nn.BatchNorm1d(out_dim)
        self.pool6 = nn.AvgPool1d(kernel_size=32, stride=1)
        
        # Define fully connected layer:
        self.fc = nn.Linear(in_features=192, out_features=11)
        self.softmax = nn.Softmax(11)

        # Initialize weights
        torch_init.xavier_normal_(self.conv1.weight)
        torch_init.xavier_normal_(self.conv2.weight)
        torch_init.xavier_normal_(self.conv3.weight)
        torch_init.xavier_normal_(self.conv4.weight)
        torch_init.xavier_normal_(self.conv5.weight)
        torch_init.xavier_normal_(self.conv6.weight)
        torch_init.xavier_normal_(self.fc.weight)


    def forward(self, batch):
        """Pass the batch of images through each layer of the network, applying
        non-linearities after each layer.

        Params:
        -------
        - batch: (Tensor) An input batch of images
        Returns:
        --------
        - logits: (Variable) The output of the network
        """

        # Apply convolutions
        batch = self.pool1(func.relu(self.conv1_norm(self.conv1(batch))))
        print('Out1 shape: ', batch.shape)
        batch = self.pool2(func.relu(self.conv2_norm(self.conv2(batch))))
        print('Out2 shape: ', batch.shape)
        batch = self.pool3(func.relu(self.conv3_norm(self.conv3(batch))))
        print('Out3 shape: ', batch.shape)
        batch = self.pool4(func.relu(self.conv4_norm(self.conv4(batch))))
        print('Out4 shape: ', batch.shape)
        batch = self.pool5(func.relu(self.conv5_norm(self.conv5(batch))))
        print('Out5 shape: ', batch.shape)
        batch = self.pool6(func.relu(self.conv6_norm(self.conv6(batch))))
        print('Out6 shape: ', batch.shape)
        
        # Reshape the output of the conv3 to pass to fully-connected layer
        batch = batch.view(-1, self.num_flat_features(batch))
        print(batch.shape)
        
        # Connect the reshaped features of the pooled conv4 to fc1
        batch = (self.fc(batch))
        print(batch.shape)

        # Return the class predictions
        return batch

    def num_flat_features(self, inputs):

        # Get the dimensions of the layers excluding the inputs
        size = inputs.size()[1:]
        # Track the number of features
        num_features = 1

        for s in size:
            num_features *= s
        return num_features

In [52]:
model = ModulationPredictionCNN()
signal = torch.rand(1, 2, 1024)
output = model(signal)

Out1 shape:  torch.Size([1, 16, 512])
Out2 shape:  torch.Size([1, 24, 256])
Out3 shape:  torch.Size([1, 36, 128])
Out4 shape:  torch.Size([1, 48, 64])
Out5 shape:  torch.Size([1, 64, 32])
Out6 shape:  torch.Size([1, 96, 2])
torch.Size([1, 192])
torch.Size([1, 11])
